In [213]:
import pandas as pd
import numpy as np
#models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#success metric
from sklearn.metrics import accuracy_score 

import matplotlib.pyplot as plt
%matplotlib inline

In [127]:
df=pd.read_csv("C:/Users/piotr/OneDrive/Pulpit/names.csv")
df.head()


,name,gender
0,Abdon,m
1,Abel,m
2,Abercjusz,m
3,Abraham,m
4,Absalon,m


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 2 columns):
name      1705 non-null object
gender    1705 non-null object
dtypes: object(2)
memory usage: 26.7+ KB


In [129]:
df.gender.value_counts().

m    1033
f     672
Name: gender, dtype: int64

Plik ma 1705 linii - obserwacji
Obiekty posiadają 2 cechy, name oraz gender.
nie ma braków w danych
zmienna gender posiada dwa warianty m oraz f
zmienna gender jest docelowa.
Klasyfikacja binarna mająca stwierdzić jakiej płci jest imię
1033 męskich imion, 672 imiona żeńskie


In [130]:
#Przekształcamy f i m w binarne odpowiedniki
def transform_string_into_number(string):
    return int(string == 'm')#Zwraca wartość int boleeanu w wyrażeniu
    
df['gender'].head().map( transform_string_into_number )
#Dodajemy wyniki jako nową kolumnę target
df['target'] = df['gender'].map(lambda x: int(x == 'm'))
df.head(10)


,name,gender,target
0,Abdon,m,1
1,Abel,m,1
2,Abercjusz,m,1
3,Abraham,m,1
4,Absalon,m,1
5,Achacjusz,m,1
6,Achacy,m,1
7,Achilles,m,1
8,Ada,f,0
9,Adalbert,m,1


Chcemy przewidzieć czy imię jest męskie lub żeńskie. Dla przykładu została wybrana długość imienia.
Dodajemy kolumnę przechowującą długość imienia:

pomysł: zakończenie na a, długość, i jakie imiona są obupłciowe, 

In [131]:
df['len_name'] = df['name'].map(lambda x: len(x))

Modele:
1.Podstawowy - wynik 0.5354
2.Regresja Logistyczna - wynik 0.6058
3.Drzewo decyzyjne - wynik 0.6093

In [132]:
X = df[ ['len_name'] ].values
y = df['target'].values

In [141]:
model = DummyClassifier()
model.fit(X, y)
y_pred = model.predict(X)
accuracy_score(y, y_pred)

0.5313782991202346

In [135]:
X = df[ ['len_name'] ].values
y = df['target'].values
model2 = LogisticRegression()
model2.fit(X, y)  
y_pred = model2.predict(X)
accuracy_score(y, y_pred)

0.6058651026392962

In [197]:
X_2 = df[ ['len_name'] ].values
y_2 = df['target'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2,train_size=0.7, test_size=0.3, random_state=1)

# Część własna

In [163]:
model3 = DecisionTreeClassifier()
model3.fit(X_train,y_train)
model3.score(X_test,y_test)

0.61328125

Tworzę nowe X i y, mimo, że nie powinienem aby móc wrócić do poprzednich modeli szybciej

In [ ]:
Zmienne do dodania:
Czy imię jest zakończone na a
czy imię jest wyjątkiem od reguły (kończy się na a, jest męskie)


In [164]:
df['end_name'] = df['name'].str.endswith("a")
df["end_name"] = df["end_name"].astype(int)
df.head(10)


,name,gender,target,len_name,target2,end_name
0,Abdon,m,1,5,False,0
1,Abel,m,1,4,False,0
2,Abercjusz,m,1,9,False,0
3,Abraham,m,1,7,False,0
4,Absalon,m,1,7,False,0
5,Achacjusz,m,1,9,False,0
6,Achacy,m,1,6,False,0
7,Achilles,m,1,8,False,0
8,Ada,f,0,3,True,1
9,Adalbert,m,1,8,False,0


In [217]:
df["exc"]=np.where((df["end_name"]==df["target"]),1,0)
df.head(10)

,name,gender,target,len_name,target2,end_name,exc
0,Abdon,m,1,5,False,0,0
1,Abel,m,1,4,False,0,0
2,Abercjusz,m,1,9,False,0,0
3,Abraham,m,1,7,False,0,0
4,Absalon,m,1,7,False,0,0
5,Achacjusz,m,1,9,False,0,0
6,Achacy,m,1,6,False,0,0
7,Achilles,m,1,8,False,0,0
8,Ada,f,0,3,True,1,0
9,Adalbert,m,1,8,False,0,0


In [234]:
X_3 = df[ ['len_name',"end_name","exc"] ].values
y_3 = df['target'].values
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_3, y_3,train_size=0.7, test_size=0.3,random_state=1)

In [239]:
wynik=0
for i in range(0,10000):
    model.fit(X_3, y_3)
    y_pred3 = model.predict(X_3)
    scr=accuracy_score(y_3, y_pred3)
    if scr>wynik:
        wynik=scr
print(wynik)

0.5671554252199413


Najwyższy uzyskany przeze mnie wynik dla modelu pierwszego to 0.5747800586510264. Lepiej niż wcześniej, ale wciąż jest to mniej niż 60% skuteczności

In [240]:
wynik=0
for i in range(0,10000):
    model2.fit(X_3, y_3)
    y_pred3 = model2.predict(X_3)
    scr=accuracy_score(y_3, y_pred3)
    if scr>wynik:
        wynik=scr
print(wynik)

0.9917888563049854


Model regresji logistycznej w tym przypadku osiągnął bardzo wysoki wynik: 0.9917888563049854

In [241]:
model3.fit(X_train2,y_train2)
model3.score(X_test2,y_test2)


1.0

Model drzewa decyzyjnego osiągnął 0.994140625 dla 2 zmiennych oraz 1.0 dla 3 zmiennych przy random state =1.


In [242]:
tree=model3.predict(X_3)
accuracy_score(y_3,tree)

1.0